In [1]:
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, Convolution2D, MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import optimizers
from keras.preprocessing import sequence
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import ModelCheckpoint

In [2]:
DROPOUT = 0.5
N_CATEGORY = 3
model_input = Input(shape = (3, 227, 227))

# First convolutional Layer (96x11x11)
z = Conv2D(filters = 96, kernel_size = (11,11), strides = (4,4), activation = "relu")(model_input)
z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = BatchNormalization()(z)

# Second convolutional Layer (256x5x5)
z = ZeroPadding2D(padding = (2,2))(z)
z = Convolution2D(filters = 256, kernel_size = (5,5), strides = (1,1), activation = "relu")(z)
z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = BatchNormalization()(z)

# Rest 3 convolutional layers
z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 384, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 384, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 256, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = Flatten()(z)

z = Dense(4096, activation="relu")(z)
z = Dropout(DROPOUT)(z)

z = Dense(4096, activation="relu")(z)
z = Dropout(DROPOUT)(z)

final_dim = 1 if N_CATEGORY == 2 else N_CATEGORY
final_act = "sigmoid" if N_CATEGORY == 2 else "softmax"
model_output = Dense(final_dim, activation=final_act)(z)

model = Model(model_input, model_output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 227, 227)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 55, 55)        34944     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 96, 27, 27)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 27, 27)        108       
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 96, 31, 31)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 27, 27)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 256, 13, 13)       0         
__________

In [3]:
WEIGHT_DECAY = 0.0005
MOMENTUM = 0.9
BATCH_SIZE = 128
LEARNING_RATE = 0.01
DROPOUT = 0.5
ALPHA = 1e-4
BETA = 0.75
n = 5
k = 2

loss_metric = "binary_crossentropy" if N_CATEGORY == 2 else "categorical_crossentropy"
model.compile(loss=loss_metric, metrics=["accuracy"], 
              optimizer=optimizers.SGD(lr=LEARNING_RATE, momentum=MOMENTUM, decay = WEIGHT_DECAY))

In [4]:
# Data augmentation
train_datagen = ImageDataGenerator(
          rotation_range=40, width_shift_range=0.2,
          height_shift_range=0.2, rescale=1./200,
          shear_range=0.2, zoom_range=0.2,
          horizontal_flip=True, fill_mode='nearest'
           )

validate_datagen = ImageDataGenerator(rescale=1./200)

train_data_dir = '/media/saqib/Mjolnir/EmotiW/Data/Heatmaps/Simple-Gaussians/train'
validation_data_dir = '/media/saqib/Mjolnir/EmotiW/Data/Heatmaps/Simple-Gaussians/validation'
nb_train_samples = 1117 + 1150 + 1241
nb_validation_samples = 528 + 689 + 761
epochs = 100
batch_size = 40

train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(227, 227),
                    batch_size=batch_size,
                    class_mode='categorical'
                    )

validation_generator = validate_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(227, 227),
                            batch_size=batch_size,
                            class_mode='categorical'
                            )

# Checkpoints
filepath = 'AlexNet-Downsized-Checkpoints/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

Found 3508 images belonging to 3 classes.
Found 1978 images belonging to 3 classes.


In [ ]:
model.fit_generator(train_generator, 
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,
                    callbacks=callbacks_list
                   )

Epoch 1/100
87/87 [==============================] - 42s - loss: 1.1485 - acc: 0.3526 - val_loss: 1.0727 - val_acc: 0.4485
Epoch 2/100
87/87 [==============================] - 42s - loss: 1.0890 - acc: 0.3823 - val_loss: 1.0726 - val_acc: 0.4701
Epoch 3/100
87/87 [==============================] - 39s - loss: 1.0761 - acc: 0.4059 - val_loss: 1.0633 - val_acc: 0.4474
Epoch 4/100
87/87 [==============================] - 39s - loss: 1.0651 - acc: 0.4354 - val_loss: 1.0794 - val_acc: 0.3896
Epoch 5/100
87/87 [==============================] - 39s - loss: 1.0596 - acc: 0.4428 - val_loss: 1.0688 - val_acc: 0.4613
Epoch 6/100
87/87 [==============================] - 42s - loss: 1.0356 - acc: 0.4553 - val_loss: 0.9787 - val_acc: 0.5346
Epoch 7/100
87/87 [==============================] - 39s - loss: 1.0338 - acc: 0.4487 - val_loss: 1.0285 - val_acc: 0.4840
Epoch 8/100
87/87 [==============================] - 39s - loss: 1.0332 - acc: 0.4570 - val_loss: 0.9678 - val_acc: 0.5294
Epoch 9/100
87/8

87/87 [==============================] - 40s - loss: 0.9437 - acc: 0.5445 - val_loss: 0.9329 - val_acc: 0.5418
Epoch 67/100
87/87 [==============================] - 39s - loss: 0.9424 - acc: 0.5347 - val_loss: 0.9263 - val_acc: 0.5449
Epoch 68/100
87/87 [==============================] - 39s - loss: 0.9502 - acc: 0.5341 - val_loss: 0.9347 - val_acc: 0.5335
Epoch 69/100
87/87 [==============================] - 40s - loss: 0.9474 - acc: 0.5236 - val_loss: 0.9163 - val_acc: 0.5459
Epoch 70/100
87/87 [==============================] - 40s - loss: 0.9419 - acc: 0.5344 - val_loss: 0.9472 - val_acc: 0.5382
Epoch 71/100
87/87 [==============================] - 40s - loss: 0.9438 - acc: 0.5326 - val_loss: 0.9177 - val_acc: 0.5495
Epoch 72/100
87/87 [==============================] - 40s - loss: 0.9442 - acc: 0.5351 - val_loss: 0.9486 - val_acc: 0.5268
Epoch 73/100
87/87 [==============================] - 40s - loss: 0.9434 - acc: 0.5300 - val_loss: 0.9517 - val_acc: 0.5273
Epoch 74/100
87/87 [=

In [7]:
model.evaluate_generator(train_generator, 3508)

[0.92001305864420579, 0.5537828929330173]

In [8]:
model.evaluate_generator(validation_generator, 1978)

[0.93655409157763714, 0.53443251351591636]

In [9]:
model.metrics_names

['loss', 'acc']